### Grid search
To find the optimal hyperparameter $\lambda$ and the optimal model complexity for the Ridge and Lasso models, we do a grid search over several candidates for $\lambda$ and complexity, comparing the mean squared error (MSE) for the models, selecting the model with the lowest MSE. We train the models using bootstrapping.

In [1]:
import numpy as np
import json

from data_gen import FrankeDataGen, TerrainDataGen
from data_handling import DataHandler
from reg_models import RidgeModel, LassoModel

import warnings
warnings.filterwarnings('ignore')

In [2]:
degrees = [1,2,3,4,5,6,7,8,9,10,11]
lmbdas = np.array([np.array([1,2,3,4,5,6,7,8,9])*10**(n) for n in range(-5, 1)]).ravel()
np.random.seed(42)
n_samples = 200

In [3]:
def ridge_grid_search(handler):
    
    model = RidgeModel(handler)
    min_MSE = 10000
    best_lambda = 0
    best_deg = 0

    for deg in degrees:
        for lmbda in lmbdas:
            MSE = model.MSE_bootstrap(n_samples=n_samples, degree=deg, ridge_lambda=lmbda)
            if MSE < min_MSE:
                min_MSE = MSE
                best_lambda = lmbda
                best_deg = deg

    new_degs = [best_deg-1, best_deg, best_deg+1]
    new_lmbdas = np.linspace(best_lambda*(4/5), best_lambda*(6/5), 40)
    print(f"Best MSE: {min_MSE}, at degree: {best_deg} and lambda: {best_lambda}")

    for deg in new_degs:
        for lmbda in new_lmbdas:
            MSE = model.MSE_bootstrap(n_samples=n_samples, degree=deg, ridge_lambda=lmbda)
            if MSE < min_MSE:
                min_MSE = MSE
                best_lambda = lmbda
                best_deg = deg

    print(f"Best MSE: {min_MSE}, at degree: {best_deg} and lambda: {best_lambda}")
    return best_lambda, best_deg

In [4]:
def lasso_grid_search(handler):

    model = LassoModel(handler)
    min_MSE = 10000
    best_lambda = 0
    best_deg = 0

    for deg in degrees:
        for lmbda in lmbdas:
            MSE = model.MSE_bootstrap(n_samples=n_samples, degree=deg, lasso_lambda=lmbda)
            if MSE < min_MSE:
                min_MSE = MSE
                best_lambda = lmbda
                best_deg = deg
    
    new_degs = [best_deg-1, best_deg, best_deg+1]
    new_lmbdas = np.linspace(best_lambda*(4/5), best_lambda*(6/5), 40)
    print(f"Best MSE: {min_MSE}, at degree: {best_deg} and lambda: {best_lambda}")

    for deg in new_degs:
        for lmbda in new_lmbdas:
            MSE = model.MSE_bootstrap(n_samples=n_samples, degree=deg, ridge_lambda=lmbda)
            if MSE < min_MSE:
                min_MSE = MSE
                best_lambda = lmbda
                best_deg = deg

    print(f"Best MSE: {min_MSE}, at degree: {best_deg} and lambda: {best_lambda}")
    return best_lambda, best_deg

##### Franke Function with Noise

In [5]:
data = FrankeDataGen(data_points=41, noise=True)
handler = DataHandler(data=data, test_size=0.2)

In [6]:
# ridge 
best_lambda, best_deg = ridge_grid_search(handler)

best_params_ridge_franke = {"best_lambda": best_lambda, "best_degree": best_deg}

Best MSE: 0.09033399355294418, at degree: 7 and lambda: 0.02
Best MSE: 0.09029019676571981, at degree: 7 and lambda: 0.024


In [7]:
# lasso 
best_deg, best_lambda = lasso_grid_search(handler)

best_params_lasso_franke = {"best_lambda": best_lambda, "best_degree": best_deg}

Best MSE: 0.0959264292822544, at degree: 8 and lambda: 2e-05
Best MSE: 0.09575126712835873, at degree: 7 and lambda: 1.7230769230769234e-05


##### Terrain Data

In [8]:
data_terrain = TerrainDataGen(data_points=41)
handler_terrain = DataHandler(data=data_terrain, test_size=0.2)

In [9]:
# ridge 
best_lambda, best_deg = ridge_grid_search(handler_terrain)

best_params_ridge_terrain = {"best_lambda": best_lambda, "best_degree": best_deg}

Best MSE: 156.86322530887014, at degree: 4 and lambda: 3.0000000000000004e-05
Best MSE: 156.86322530887014, at degree: 4 and lambda: 3.0000000000000004e-05


In [10]:
# lasso 
best_lambda, best_deg = lasso_grid_search(handler_terrain)

best_params_lasso_terrain = {"best_lambda": best_lambda, "best_degree": best_deg}

Best MSE: 155.90596195500808, at degree: 5 and lambda: 0.0002
Best MSE: 150.23321351904164, at degree: 6 and lambda: 0.00022974358974358976


In [11]:
params = {
    "best_params_ridge_franke": best_params_ridge_franke,
    "best_params_lasso_franke": best_params_lasso_franke,
    "best_params_ridge_terrain": best_params_ridge_terrain,
    "best_params_lasso_terrain": best_params_lasso_terrain
}

with open("best_params.json", "w") as f:
    json.dump(params, f)

    